#### Key setup

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

#### Imports

In [3]:
from langchain_groq import ChatGroq
import requests
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

#### Search Tool

In [4]:
search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke("top news in india today?")
print(result)

India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Headlines today: Get news headlines from India and around the world. Today's breaking news on politics, sports, entertainment, business, life style and many more on Times of India Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle and more at CNBC TV18. Join Us. ... India enters top 100 in global SDG rankings for first time. Jun 25, 2025 7:26 PM 3 Min Read. Entertainment. 50 Years of Emergency: Aandhi to Nasbandi - the films that were silenced and ... Stay informed with breaking news and latest news on politics, business, entertainment sports, science, technology along with news updates from around the world. Reuters.com is your online source for the latest India news stories and current events, ensuring our readers up to da

#### Making a additional tool

In [5]:
@tool
def get_weather_data(city: str) -> str:
    """This fuction fetches the current weather data from a given city"""
    url = f"http://api.weatherstack.com/current?access_key=6042591fc1c28b3c30ec4e2aeac3fe79&query={city}"
    response = requests.get(url)
    return response.json()

#### LLM 

In [6]:

model = ChatGroq(model="llama-3.3-70b-versatile", api_key=api_key)
model.invoke("hello")

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.036363636, 'prompt_time': 0.00153465, 'queue_time': 0.055559049, 'total_time': 0.037898286}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--94c3da09-4789-442d-9a44-4bef4d153866-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

#### Agent imports

In [7]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub # hub has pre made prompts 

#### Pulling Pre-made Prompt from HUB

In [8]:
#pull ReAct prompt from langchain hub
prompt = hub.pull("hwchase17/react") # pull the standard ReAct agent prompt ( reasoning + action)

#### Create ReAct agent manually with the pulled prompt

In [ ]:
agent = create_react_agent(
    llm = model, 
    tools=[search_tool, get_weather_data], # which tools agent has access to
    prompt=prompt
)

###### Agent does the planning and reasoning and agent executer does the work on ground
#### wrap agent with Agent Executer

In [ ]:
agent_executer = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True # we will see what agent thinks 
)

#### Invoke the Agent Executor

In [ ]:
response = agent_executer.invoke({"input":"Find Capital of Uttar Pradesh, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
To find ways to reach Varanasi from Delhi, I should first consider the common modes of transportation available in India, such as trains, flights, and buses. However, to provide more specific and up-to-date information, I should search for the latest options and schedules.

Action: duckduckgo_search
Action Input: Varanasi to Delhi transport options

(I'll wait for the search results to proceed further)Varanasi to Delhi Bus tickets Are you looking for a convenient and safe mode of transportation from Varanasi to Delhi? Look no further as zingbus offers a comfortable commute to Delhi. The first bus starts from Varanasi at 3:45 PM. The minimum bus ticket booking fare is ₹ 600 and goes up to ₹ 1237 depending on the bus type and timings. Varanasi to Delhi bus booking starts @ ₹1049 - Get best ticket price on sleeper, volvo, private, government bus ticket booking from Varanasi to Delhi. Use Coupon Code IDBIRIDE and get discount upto ₹500 and earn rewar

In [12]:
print(response["output"])

There are three ways to reach Varanasi from Delhi: by bus, which is the most affordable option and takes around 13-14 hours; by flight, which is the fastest option and takes around 2h 26m; and by train, which takes around 12-14 hours and offers various classes of travel.
